### Test with created rule/ normal allocation not in recession / cash in recession 

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

In [2]:
### Load Asset data
df = pd.read_csv('data_for_test.csv')
df.drop('Unnamed: 0', inplace = True, axis = 1)
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%y")
ch = df[['Date', 'S&P', 'HS','Gold', 'FTSE', 'DJ', 'AGG']]
ch.head()

,Date,S&P,HS,Gold,FTSE,DJ,AGG
0,1983-12-30,NaN,NaN,NaN,NaN,NaN,NaN
1,1984-01-31,-0.009216,0.259949,-0.022620,0.063000,-0.030239,0.020617
2,1984-02-29,-0.038859,-0.039088,0.054849,-0.021355,-0.054032,-0.005126
3,1984-03-30,0.013498,-0.042396,-0.014585,0.069403,0.008886,-0.011224
4,1984-04-30,0.005466,0.022358,-0.032690,0.023191,0.005031,-0.001961


In [3]:
### Load Rule Data
m3 = pd.read_csv('10year_minus.csv')
m3['Dates'] = pd.to_datetime(m3.Dates)
df_all = pd.concat([ch, m3], axis =1)
df_all.drop('Dates', inplace = True,axis = 1)
df_all

,Date,S&P,HS,Gold,FTSE,DJ,AGG,T10Y3M,Diff_2_10,in_rec,out_rec,rec
0,1983-12-30,NaN,NaN,NaN,NaN,NaN,NaN,2.410476,1.03,2,0,0
1,1984-01-31,-0.009216,0.259949,-0.022620,0.063000,-0.030239,0.020617,2.382632,1.05,2,0,0
2,1984-02-29,-0.038859,-0.039088,0.054849,-0.021355,-0.054032,-0.005126,2.429545,1.01,2,0,0
3,1984-03-30,0.013498,-0.042396,-0.014585,0.069403,0.008886,-0.011224,2.560500,0.94,2,0,0
4,1984-04-30,0.005466,0.022358,-0.032690,0.023191,0.005031,-0.001961,3.185455,0.94,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
424,2019-04-30,0.039313,0.022297,-0.010113,0.019100,0.025618,0.000256,-0.008182,0.19,0,0,0
425,2019-05-31,-0.065778,-0.094212,0.010333,-0.034578,-0.066855,0.017752,-0.145500,0.26,0,0,0
426,2019-06-28,0.068930,0.061021,0.087569,0.036852,0.071929,0.012556,-0.085909,0.22,0,0,0
427,2019-07-31,0.013128,-0.026797,0.013165,0.021702,0.009936,0.002201,-0.364091,0.06,0,0,0


In [4]:
### Load Allecation Data
df2 = pd.read_csv('index_al.csv').set_index('Index')
df2

,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,115,116,117,118,119,120
Index,,,,,,,,,,,,,,,,,,,,,
SP,0.175,0.1744,0.1738,0.1733,0.1727,0.1721,0.1715,0.1709,0.1703,0.1698,...,0.1067,0.1060,0.1053,0.1047,0.104,0.1033,0.1027,0.1020,0.1013,0.1007
DJ,0.175,0.1744,0.1738,0.1733,0.1727,0.1721,0.1715,0.1709,0.1703,0.1698,...,0.1067,0.1060,0.1053,0.1047,0.104,0.1033,0.1027,0.1020,0.1013,0.1007
HS,0.115,0.1147,0.1143,0.1140,0.1137,0.1133,0.1130,0.1127,0.1123,0.1120,...,0.0700,0.0695,0.0690,0.0685,0.068,0.0675,0.0670,0.0665,0.0660,0.0655
FTSE,0.115,0.1147,0.1143,0.1140,0.1137,0.1133,0.1130,0.1127,0.1123,0.1120,...,0.0700,0.0695,0.0690,0.0685,0.068,0.0675,0.0670,0.0665,0.0660,0.0655
AGG,0.210,0.2109,0.2118,0.2128,0.2137,0.2146,0.2155,0.2164,0.2173,0.2183,...,0.3233,0.3245,0.3257,0.3268,0.328,0.3292,0.3303,0.3315,0.3327,0.3338
Gold,0.210,0.2109,0.2118,0.2128,0.2137,0.2146,0.2155,0.2164,0.2173,0.2183,...,0.3233,0.3245,0.3257,0.3268,0.328,0.3292,0.3303,0.3315,0.3327,0.3338


### 120 Months

In [5]:
data = pd.DataFrame([], columns = ['Start_date','Month', 'Date', 'Start_Value', 'Return_Value','Recession_Count'])
dd_list = df_all.Date.tolist()
for ww in dd_list:
    if ww > pd.to_datetime('2009-08-31'):
        break
    sv = 100000.00
    dd = df_all[df_all.Date >= ww].reset_index().drop('index', axis = 1)
    sd = dd.iloc[0][0]
    rec_count = 0
    print(sd.date())
    for i in range(1,121):
        sv2 = sv
        col = i - 1
        sp_value = round(sv * df2.iloc[0,col],2)
        dj_value = round(sv * df2.iloc[1,col],2)
        hs_value = round(sv * df2.iloc[2,col],2)
        ft_value = round(sv * df2.iloc[3,col],2)
        agg_value = round(sv * df2.iloc[4,col],2)
        gold_value = round(sv * df2.iloc[5,col],2)
        if dd.iloc[i][11] == 1:
            dt = dd.iloc[i][0]
            rec_count += 1
            mm = 'Month ' + str(i)
            data = data.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                                'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)
            continue 

        dt = dd.iloc[i][0]
        sp500 = dd.iloc[i][1]
        hs = dd.iloc[i][2]
        gold = dd.iloc[i][3]
        ft = dd.iloc[i][4]
        dj = dd.iloc[i][5]
        agg = dd.iloc[i][6]

        sp500_cash = round(sp_value * (1 + sp500) ,2)

        hs_cash = round(hs_value * (1 + hs) ,2)

        gold_cash = round(gold_value * (1 + gold) ,2)

        ft_cash = round(ft_value * (1 + ft) ,2)

        dj_cash = round(dj_value * (1 + dj) ,2)

        agg_cash = round(agg_value * (1 + agg) ,2)

        sv = round(sum([agg_cash, dj_cash, ft_cash, gold_cash, hs_cash, sp500_cash]),2)
        mm = 'Month ' + str(i)
        data = data.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                            'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)

1983-12-30
1984-01-31
1984-02-29
1984-03-30
1984-04-30
1984-05-31
1984-06-29
1984-07-31
1984-08-31
1984-09-28
1984-10-31
1984-11-30
1984-12-31
1985-01-31
1985-02-28
1985-03-29
1985-04-30
1985-05-31
1985-06-28
1985-07-31
1985-08-30
1985-09-30
1985-10-31
1985-11-29
1985-12-31
1986-01-31
1986-02-28
1986-03-31
1986-04-30
1986-05-30
1986-06-30
1986-07-31
1986-08-29
1986-09-30
1986-10-31
1986-11-28
1986-12-31
1987-01-30
1987-02-27
1987-03-31
1987-04-30
1987-05-29
1987-06-30
1987-07-31
1987-08-31
1987-09-30
1987-10-30
1987-11-30
1987-12-31
1988-01-29
1988-02-29
1988-03-31
1988-04-29
1988-05-31
1988-06-30
1988-07-29
1988-08-31
1988-09-30
1988-10-31
1988-11-30
1988-12-30
1989-01-31
1989-02-28
1989-03-31
1989-04-28
1989-05-31
1989-06-30
1989-07-31
1989-08-31
1989-09-29
1989-10-31
1989-11-30
1989-12-29
1990-01-31
1990-02-28
1990-03-30
1990-04-30
1990-05-31
1990-06-29
1990-07-31
1990-08-31
1990-09-28
1990-10-31
1990-11-30
1990-12-31
1991-01-31
1991-02-28
1991-03-29
1991-04-30
1991-05-31
1991-06-28

In [6]:
### results 
data

,Start_date,Month,Date,Start_Value,Return_Value,Recession_Count
0,1983-12-30,Month 1,1984-01-31,100000.00,102981.40,0
1,1983-12-30,Month 2,1984-02-29,102981.40,101679.07,0
2,1983-12-30,Month 3,1984-03-30,101679.07,101812.37,0
3,1983-12-30,Month 4,1984-04-30,101812.37,101795.86,0
4,1983-12-30,Month 5,1984-05-31,101795.86,97042.56,0
...,...,...,...,...,...,...
37075,2009-08-31,Month 116,2019-04-30,174820.59,175914.43,0
37076,2009-08-31,Month 117,2019-05-31,175914.43,173632.14,0
37077,2009-08-31,Month 118,2019-06-28,173632.14,183020.03,0
37078,2009-08-31,Month 119,2019-07-31,183020.03,184321.75,0


In [7]:
data.to_csv('rule_120months.csv')

In [8]:
### Mean Return
print('Mean Return 120 Month:', round(data[data['Month'] == 'Month 120']['Return_Value'].mean(),2))

Mean Return 120 Month: 208554.68


### 60-65 

In [9]:
data2 = pd.DataFrame([], columns = ['Start_date','Month', 'Date', 'Start_Value', 'Return_Value','Recession_Count'])
dd_list = df_all.Date.tolist()
for ww in dd_list:
    if ww > pd.to_datetime('2014-08-31'):
        break
    sv = 100000.00
    dd = df_all[df_all.Date >= ww].reset_index().drop('index', axis = 1)
    sd = dd.iloc[0][0]
    rec_count = 0
    print(sd.date())
    for i in range(1,61):
        sv2 = sv
        col = i - 1
        sp_value = round(sv * df2.iloc[0,col],2)
        dj_value = round(sv * df2.iloc[1,col],2)
        hs_value = round(sv * df2.iloc[2,col],2)
        ft_value = round(sv * df2.iloc[3,col],2)
        agg_value = round(sv * df2.iloc[4,col],2)
        gold_value = round(sv * df2.iloc[5,col],2)
        if dd.iloc[i][11] == 1:
            dt = dd.iloc[i][0]
            rec_count += 1
            mm = 'Month ' + str(i)
            data2 = data2.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                                'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)
            continue 

        dt = dd.iloc[i][0]
        sp500 = dd.iloc[i][1]
        hs = dd.iloc[i][2]
        gold = dd.iloc[i][3]
        ft = dd.iloc[i][4]
        dj = dd.iloc[i][5]
        agg = dd.iloc[i][6]

        sp500_cash = round(sp_value * (1 + sp500) ,2)

        hs_cash = round(hs_value * (1 + hs) ,2)

        gold_cash = round(gold_value * (1 + gold) ,2)

        ft_cash = round(ft_value * (1 + ft) ,2)

        dj_cash = round(dj_value * (1 + dj) ,2)

        agg_cash = round(agg_value * (1 + agg) ,2)

        sv = round(sum([agg_cash, dj_cash, ft_cash, gold_cash, hs_cash, sp500_cash]),2)
        mm = 'Month ' + str(i)
        data2 = data2.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                            'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)

1983-12-30
1984-01-31
1984-02-29
1984-03-30
1984-04-30
1984-05-31
1984-06-29
1984-07-31
1984-08-31
1984-09-28
1984-10-31
1984-11-30
1984-12-31
1985-01-31
1985-02-28
1985-03-29
1985-04-30
1985-05-31
1985-06-28
1985-07-31
1985-08-30
1985-09-30
1985-10-31
1985-11-29
1985-12-31
1986-01-31
1986-02-28
1986-03-31
1986-04-30
1986-05-30
1986-06-30
1986-07-31
1986-08-29
1986-09-30
1986-10-31
1986-11-28
1986-12-31
1987-01-30
1987-02-27
1987-03-31
1987-04-30
1987-05-29
1987-06-30
1987-07-31
1987-08-31
1987-09-30
1987-10-30
1987-11-30
1987-12-31
1988-01-29
1988-02-29
1988-03-31
1988-04-29
1988-05-31
1988-06-30
1988-07-29
1988-08-31
1988-09-30
1988-10-31
1988-11-30
1988-12-30
1989-01-31
1989-02-28
1989-03-31
1989-04-28
1989-05-31
1989-06-30
1989-07-31
1989-08-31
1989-09-29
1989-10-31
1989-11-30
1989-12-29
1990-01-31
1990-02-28
1990-03-30
1990-04-30
1990-05-31
1990-06-29
1990-07-31
1990-08-31
1990-09-28
1990-10-31
1990-11-30
1990-12-31
1991-01-31
1991-02-28
1991-03-29
1991-04-30
1991-05-31
1991-06-28

In [10]:
### results 
data2

,Start_date,Month,Date,Start_Value,Return_Value,Recession_Count
0,1983-12-30,Month 1,1984-01-31,100000.00,102981.40,0
1,1983-12-30,Month 2,1984-02-29,102981.40,101679.07,0
2,1983-12-30,Month 3,1984-03-30,101679.07,101812.37,0
3,1983-12-30,Month 4,1984-04-30,101812.37,101795.86,0
4,1983-12-30,Month 5,1984-05-31,101795.86,97042.56,0
...,...,...,...,...,...,...
22135,2014-08-29,Month 56,2019-04-30,121143.85,122441.93,0
22136,2014-08-29,Month 57,2019-05-31,122441.93,119486.49,0
22137,2014-08-29,Month 58,2019-06-28,119486.49,126157.70,0
22138,2014-08-29,Month 59,2019-07-31,126157.70,127042.51,0


In [11]:
data2.to_csv('rule60_65_60months.csv')

In [12]:
### Mean Return 
print('Mean Return 60-65:' , round(data2[data2.Month == 'Month 60']['Return_Value'].mean(),2))

Mean Return 60-65: 146189.47


### 65-70 

In [13]:
data3 = pd.DataFrame([], columns = ['Start_date','Month', 'Date', 'Start_Value', 'Return_Value','Recession_Count'])
dd_list = df_all.Date.tolist()
for ww in dd_list:
    if ww > pd.to_datetime('2014-08-31'):
        break
    sv = 100000.00
    dd = df_all[df_all.Date >= ww].reset_index().drop('index', axis = 1)
    sd = dd.iloc[0][0]
    rec_count = 0
    print(sd.date())
    for i in range(1,61):
        sv2 = sv
        col = i + 59
        sp_value = round(sv * df2.iloc[0,col],2)
        dj_value = round(sv * df2.iloc[1,col],2)
        hs_value = round(sv * df2.iloc[2,col],2)
        ft_value = round(sv * df2.iloc[3,col],2)
        agg_value = round(sv * df2.iloc[4,col],2)
        gold_value = round(sv * df2.iloc[5,col],2)
        if dd.iloc[i][11] == 1:
            dt = dd.iloc[i][0]
            rec_count += 1
            mm = 'Month ' + str(i)
            data3 = data3.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                                'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)
            continue 

        dt = dd.iloc[i][0]
        sp500 = dd.iloc[i][1]
        hs = dd.iloc[i][2]
        gold = dd.iloc[i][3]
        ft = dd.iloc[i][4]
        dj = dd.iloc[i][5]
        agg = dd.iloc[i][6]

        sp500_cash = round(sp_value * (1 + sp500) ,2)

        hs_cash = round(hs_value * (1 + hs) ,2)

        gold_cash = round(gold_value * (1 + gold) ,2)

        ft_cash = round(ft_value * (1 + ft) ,2)

        dj_cash = round(dj_value * (1 + dj) ,2)

        agg_cash = round(agg_value * (1 + agg) ,2)

        sv = round(sum([agg_cash, dj_cash, ft_cash, gold_cash, hs_cash, sp500_cash]),2)
        mm = 'Month ' + str(i)
        data3 = data3.append({'Start_date':sd,'Month':mm, 'Date': dt, 'Start_Value':sv2, 
                            'Return_Value': sv ,'Recession_Count': rec_count},ignore_index = True)

1983-12-30
1984-01-31
1984-02-29
1984-03-30
1984-04-30
1984-05-31
1984-06-29
1984-07-31
1984-08-31
1984-09-28
1984-10-31
1984-11-30
1984-12-31
1985-01-31
1985-02-28
1985-03-29
1985-04-30
1985-05-31
1985-06-28
1985-07-31
1985-08-30
1985-09-30
1985-10-31
1985-11-29
1985-12-31
1986-01-31
1986-02-28
1986-03-31
1986-04-30
1986-05-30
1986-06-30
1986-07-31
1986-08-29
1986-09-30
1986-10-31
1986-11-28
1986-12-31
1987-01-30
1987-02-27
1987-03-31
1987-04-30
1987-05-29
1987-06-30
1987-07-31
1987-08-31
1987-09-30
1987-10-30
1987-11-30
1987-12-31
1988-01-29
1988-02-29
1988-03-31
1988-04-29
1988-05-31
1988-06-30
1988-07-29
1988-08-31
1988-09-30
1988-10-31
1988-11-30
1988-12-30
1989-01-31
1989-02-28
1989-03-31
1989-04-28
1989-05-31
1989-06-30
1989-07-31
1989-08-31
1989-09-29
1989-10-31
1989-11-30
1989-12-29
1990-01-31
1990-02-28
1990-03-30
1990-04-30
1990-05-31
1990-06-29
1990-07-31
1990-08-31
1990-09-28
1990-10-31
1990-11-30
1990-12-31
1991-01-31
1991-02-28
1991-03-29
1991-04-30
1991-05-31
1991-06-28

In [14]:
### results 
data3

,Start_date,Month,Date,Start_Value,Return_Value,Recession_Count
0,1983-12-30,Month 1,1984-01-31,100000.00,102462.57,0
1,1983-12-30,Month 2,1984-02-29,102462.57,101907.72,0
2,1983-12-30,Month 3,1984-03-30,101907.72,101779.79,0
3,1983-12-30,Month 4,1984-04-30,101779.79,101413.76,0
4,1983-12-30,Month 5,1984-05-31,101413.76,97486.61,0
...,...,...,...,...,...,...
22135,2014-08-29,Month 56,2019-04-30,117589.33,118325.09,0
22136,2014-08-29,Month 57,2019-05-31,118325.09,116789.97,0
22137,2014-08-29,Month 58,2019-06-28,116789.97,123104.54,0
22138,2014-08-29,Month 59,2019-07-31,123104.54,123980.13,0


In [15]:
data3.to_csv('rule65_70_60months.csv')

In [16]:
### Mean Return
print('Mean Return 65-70:' , round(data3[data3.Month == 'Month 60']['Return_Value'].mean(),2))

Mean Return 65-70: 141788.32
